
## VDR chr13_42952 example -- Which component is important?

- The specific context of this variant and disease is described in this google doc:
  - https://docs.google.com/document/d/16GuSasXWX-5qwvKAX5-4VxtrbmsIu9UgrP311_viqQc/edit?usp=sharing
- This notebook would show, 
  1. Given the SNP, identify which genomic bin contains the SNP
  1. Use genomic bin squared cosine score to find the top 3 important components for the genomic bin
  1. Investigate the top component for the genomic bins
    - Use assay contribution scores to see what assays are important for the component
    - Use genomic bin contribution scores to see what other gnomic bins are important for the component
    - Explorer the results of enrichment analysis


In [1]:
% matplotlib inline

import numpy as np
import pandas as pd
import matplotlib, collections, itertools, os, re, textwrap, logging, sys
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from functools import reduce

from logging.config import dictConfig
from logging import getLogger

dictConfig(dict(
    version = 1,
    formatters = {'f': {'format': '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'}},
    handlers = {
        'h': {'class': 'logging.StreamHandler','formatter': 'f',
              'level': logging.DEBUG}},
    root = {'handlers': ['h'], 'level': logging.DEBUG,},
))

matplotlib.rc('font',**{'size':16, 'family':'sans-serif','sans-serif':['HelveticaNeue', 'Helvetica']})

logger = getLogger('notebook')


In [2]:
repo_dir=os.path.realpath(
    os.path.dirname(os.path.dirname(os.getcwd()))
)


In [3]:
data_dir=os.path.realpath(
    os.path.join(os.path.dirname(os.getcwd()), 'private_data')
)

In [4]:
enrichment_data_dir=os.path.join(repo_dir, 'enrichment', 'private_data')


In [5]:
sys.path.append(os.path.join(repo_dir, 'enrichment', 'src'))
from great import read_great_res_wrapper


In [6]:
metadata_dir=os.path.join(
    repo_dir, 'metadata'
)
metadata = pd.read_table(
    os.path.join(metadata_dir, 'sample_antibody_map_v2_with_metadata.tsv'),
    sep='\t',
)

### Step 1: SNP to genomic bin


In [7]:
genomic_bin_df=pd.read_csv(
    os.path.join(repo_dir, 'enrichment', 'private_data', 'loci_def.bed'),
    names=['chr', 'chromStart', 'chromEnd', 'name'],
    sep='\t'
)

In [8]:
genomic_bin_df[genomic_bin_df['name'] == 'chr13_42952']

,chr,chromStart,chromEnd,name
275291,chr13,42952000,42953000,chr13_42952


This means the index of the genomic bin of our interest is 275291

In [9]:
genomic_bin_idx=275291

### Step 2: Which component is important for a given genomic bin -- genomic bin squared cosine score
- Let's write our decomposition as X = UDV' where X is input feature matrix, D is diagonal singular value matrix, U is left singular vector matrix (on assay space), V is right singular vector matrix (on genomic bin space), and `'` denotes the transposition of the matrix.
- Genomic bin squared cosine score is defined as L2-normalized version of the matrix product (VD) so that any given slice for a given genomic bin has Euclidian norm of 1. 
- The interpretation of the score is it represents the relative importance of the component given a genomic bin.
- More formal definition:
  - https://docs.google.com/document/d/1YRuaIvHvjb_6SJwlml1dQDegiGlGbdfz_zN-5bneroE/edit?usp=sharing
 

#### read the decomposed matrices

In [10]:
def read_decomposed_matrix(filename, compression=None):
    if((compression is None) and (len(filename) > 3) and (filename[-3:] == '.gz')):
        compression='gzip'
    df = pd.read_csv(
        os.path.join(data_dir, filename),
        compression=compression
    )
    mat = df.iloc[:, 1:].as_matrix()
    idx = df.iloc[:, 0].as_matrix()
    return mat, idx

In [11]:
d_mat_temp, d_idx = read_decomposed_matrix(os.path.join(data_dir, 'diagonalScore.csv.gz'))
d_vec = d_mat_temp[:, 0]


In [12]:
u_mat, u_idx = read_decomposed_matrix(os.path.join(data_dir, 'uScore.csv.gz'))


In [13]:
v_mat, v_idx = read_decomposed_matrix(os.path.join(data_dir, 'vScore.csv.gz'))


In [14]:
d_vec.shape, u_mat.shape, v_mat.shape, d_idx.shape, u_idx.shape, v_idx.shape

((652,), (652, 652), (379541, 652), (652,), (652,), (379541,))

#### compute matrix products, UD and VD

In [15]:
u_dot_d = np.dot(u_mat, np.diag(d_vec))


In [16]:
v_dot_d = np.dot(v_mat, np.diag(d_vec))


In [17]:
u_dot_d.shape, v_dot_d.shape

((652, 652), (379541, 652))

#### compute normalized matrices
- v_dot_d_find_pcs: genomic bin --> which PC? genomic bin squared contribution score.
- u_dot_d_fine_pcs: assay       --> which PC? assay squared contribution score.
- v_dot_d_find_loci: PC --> which genomic bins? genomic bin contribution score.
- u_dot_d_find_assay: PC --> which assay? assay contribution score

In [18]:
v_dot_d_find_pcs = (v_dot_d ** 2 ) / (np.sum(v_dot_d ** 2, axis = 1)[:,np.newaxis])


In [19]:
u_dot_d_find_pcs = (u_dot_d ** 2 ) / (np.sum(u_dot_d ** 2, axis = 1)[:,np.newaxis])


In [20]:
v_dot_d_find_loci = (v_dot_d ** 2 ) / (np.sum(v_dot_d ** 2, axis = 0)[np.newaxis, :])


In [21]:
u_dot_d_find_assay = (u_dot_d ** 2 ) / (np.sum(u_dot_d ** 2, axis = 0)[np.newaxis, :])


#### let's identify the top 3 important components for the genomic bin

In [23]:
np.argsort(-v_dot_d_find_pcs[genomic_bin_idx, :])[:5]

array([5, 2, 8, 7, 3])

In [24]:
v_dot_d_find_pcs[genomic_bin_idx, np.argsort(-v_dot_d_find_pcs[genomic_bin_idx, :])[:5]]

array([0.27117038, 0.09215098, 0.07264342, 0.06559204, 0.04081626])

This means PC5 (0-based index) is the most important component for this bin with 27.1% of squared cosine score, PC2 is the second important one with 9.2%, etc ...

### Step 3: investigation of the components

#### PC5 (the top component)

We will investigate 

1. What assays are driving this component?
1. What genomic loci are driving this component?
1.  What are the top hits in the enrichment analysis?

In [25]:
component_idx=5

#### what assays are driving this component?

In [26]:
np.argsort(-u_dot_d_find_assay[:, component_idx])[:5]

array([188, 433,  76, 434, 431])

In [27]:
u_dot_d_find_assay[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], component_idx]

array([0.1195496 , 0.08228496, 0.05328201, 0.03843322, 0.03278715])

In [28]:
metadata.iloc[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], :]

,sample_number,antibody,Genome assembly,Antigen class,Antigen,Cell type class,Cell type,Cell type description,Processing logs,Title,...,age,treatment,genotype,lab,age.1,health state,cell_type,tissue_type,provider,sex
188,SRX1027619,NaN,hg19,No description,NaN,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"804451442,97.3,44.1,118575",ChIP-seq of Homo sapiens: H3K4me3,...,NaN,NaN,NaN,NaN,NaN,NaN,LCLs,blood,Coriell,pooled male and female
433,SRX651491,NaN,hg19,Histone,H3K4me3,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"166047212,93.9,6.4,63296",GSM1435515: LCL19238 H3K4me3; Homo sapiens; Ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,ERX329687,NaN,hg19,Unclassified,Unclassified,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"238094924,94.2,33.9,144042",Illumina HiSeq 2000 sequencing; Coordinated ef...,...,NaN,NaN,NaN,NaN,NaN,NaN,NA12878,NaN,NaN,female
434,SRX651492,NaN,hg19,Histone,H3K4me1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"112405175,94.2,1.6,121976",GSM1435516: LCL19239 H3K4me1; Homo sapiens; Ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,SRX651470,NaN,hg19,Histone,H3K4me3,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"117212731,95.4,4.0,47422",GSM1435517: LCL19239 H3K4me3; Homo sapiens; Ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### what genomic bins are driving this component?

In [29]:
np.argsort(-v_dot_d_find_loci[:, component_idx])[:5]

array([282969, 130825, 187463, 337732, 233484])

In [30]:
v_dot_d_find_loci[np.argsort(-v_dot_d_find_loci[:, component_idx])[:5], component_idx]

array([8.04405926e-05, 6.46164381e-05, 6.45142013e-05, 6.39899652e-05,
       6.37951179e-05])

These genomic bins are important for PC16. Note the genomic bin contribution scores are very small compared to assay contribution score. This is expected becasue of the large number of genomic bins in the whole-genome analysis.

#### where is our loci of interest in this ranking?

In [31]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[
    genomic_bin_idx, component_idx])

870

In [32]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[
    genomic_bin_idx, component_idx]) / v_dot_d_find_loci.shape[0]

0.0022922424718278133

It's roughly on the top 0.2 percentile.


#### results of the enrichment analysis

In [33]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'HumanPhenotypeOntology').head(10)


,# ID,Desc,BPval,BFold
0,HP:0002697,Parietal foramina,4.367679e-08,2.372433
1,HP:0004425,Flat forehead,1.568965e-06,2.735910
3,HP:0004442,Sagittal craniosynostosis,2.359962e-06,2.733096
4,HP:0002365,Hypoplasia of the brainstem,2.396735e-06,2.031767
5,HP:0010054,Abnormality of the first metatarsal,2.506458e-06,2.574793
7,HP:0006191,Deep palmar crease,4.189624e-06,2.977180
8,HP:0000557,Buphthalmos,4.706791e-06,2.287392
12,HP:0009836,Broad distal phalanx of finger,1.739019e-05,2.601329
18,HP:0003741,Congenital muscular dystrophy,3.098136e-05,2.094681
21,HP:0003535,3-Methylglutaconic aciduria,3.861840e-05,2.428799


In [34]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOBiologicalProcess').head(10)


,# ID,Desc,BPval,BFold
62,GO:0035089,establishment of apical/basal cell polarity,0.000008,2.658500
76,GO:0075733,intracellular transport of virus,0.000012,2.073909
100,GO:0051775,response to redox state,0.000021,2.764178
108,GO:0061162,establishment of monopolar cell polarity,0.000025,2.320837
113,GO:0070127,tRNA aminoacylation for mitochondrial protein ...,0.000027,4.115960
116,GO:0071624,positive regulation of granulocyte chemotaxis,0.000029,2.016287
126,GO:0090023,positive regulation of neutrophil chemotaxis,0.000034,2.020100
148,GO:0018401,peptidyl-proline hydroxylation to 4-hydroxy-L-...,0.000056,2.373342
161,GO:0001672,regulation of chromatin assembly or disassembly,0.000072,3.185283
164,GO:0006999,nuclear pore organization,0.000077,2.528835


In [35]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenotype').head(10)


,# ID,Desc,BPval,BFold
12,MP:0001771,abnormal circulating magnesium level,1.832349e-07,2.327122
19,MP:0003954,abnormal Reichert's membrane morphology,8.388949e-07,2.004437
53,MP:0002348,abnormal lymph node medulla morphology,9.682417e-06,2.903855
54,MP:0010092,increased circulating magnesium level,1.018919e-05,2.628006
59,MP:0009545,abnormal dermis papillary layer morphology,1.604188e-05,2.274394
60,MP:0010743,delayed suture closure,1.697130e-05,2.208579
65,MP:0001669,abnormal glucose absorption,2.245532e-05,2.618271
69,MP:0006210,abnormal orbit size,2.804297e-05,2.181630
77,MP:0002050,pheochromocytoma,3.830129e-05,2.589431
79,MP:0000666,decreased prostate gland duct number,4.319200e-05,2.700373


In [36]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenoSingleKO').head(10)


,# ID,Desc,BPval,BFold
5,MP:0012129,failure of blastocyst formation,2.360537e-09,2.005173
8,MP:0002663,failure to form blastocele,3.603307e-09,2.004262
23,MP:0003954,abnormal Reichert's membrane morphology,8.388949e-07,2.004437
24,MP:0001771,abnormal circulating magnesium level,1.211860e-06,2.284786
33,MP:0010743,delayed suture closure,3.083532e-06,2.694599
34,MP:0002050,pheochromocytoma,3.483130e-06,3.010810
44,MP:0002348,abnormal lymph node medulla morphology,9.682417e-06,2.903855
48,MP:0009545,abnormal dermis papillary layer morphology,1.604188e-05,2.274394
60,MP:0000275,heart hyperplasia,4.592075e-05,2.012848
69,MP:0002031,increased adrenal gland tumor incidence,6.052274e-05,2.408676


In [37]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOCellularComponent').head(10)


,# ID,Desc,BPval,BFold
5,GO:0005606,laminin-1 complex,0.000001,2.325389
7,GO:0043256,laminin complex,0.000005,2.148635
21,GO:0019031,viral envelope,0.000610,2.839336
48,GO:0031080,nuclear pore outer ring,0.002727,2.255517
54,GO:0005666,DNA-directed RNA polymerase III complex,0.004041,2.228519
58,GO:0019908,nuclear cyclin-dependent protein kinase holoen...,0.004746,2.188418
72,GO:0016461,unconventional myosin complex,0.007764,2.067299
81,GO:0005677,chromatin silencing complex,0.010734,2.111160
85,GO:0005736,DNA-directed RNA polymerase I complex,0.011807,2.494752
93,GO:0005851,eukaryotic translation initiation factor 2B co...,0.014343,2.793472


In [38]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOMolecularFunction').head(10)


,# ID,Desc,BPval,BFold
3,GO:0043022,ribosome binding,7.584495e-08,2.351977
11,GO:0031545,peptidyl-proline 4-dioxygenase activity,8.138740e-06,2.466700
14,GO:0043208,glycosphingolipid binding,2.462545e-05,2.495340
30,GO:0001056,RNA polymerase III activity,3.860479e-04,3.131341
34,GO:0005007,fibroblast growth factor-activated receptor ac...,5.214530e-04,2.250279
48,GO:0032407,MutSalpha complex binding,1.512241e-03,2.575431
60,GO:0015377,cation:chloride symporter activity,2.143232e-03,2.088132
65,GO:0047499,calcium-independent phospholipase A2 activity,2.303207e-03,3.545742
69,GO:0032404,mismatch repair complex binding,2.498749e-03,2.155177
74,GO:0016421,CoA carboxylase activity,2.932712e-03,2.029746


#### PC2 (the second component)


In [39]:
component_idx=2

#### what assays are driving this component?

In [40]:
np.argsort(-u_dot_d_find_assay[:, component_idx])[:5]

array([305, 366, 336, 321, 351])

In [41]:
u_dot_d_find_assay[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], component_idx]

array([0.0767717 , 0.07214937, 0.06740654, 0.05664575, 0.04790769])

In [42]:
metadata.iloc[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], :]

,sample_number,antibody,Genome assembly,Antigen class,Antigen,Cell type class,Cell type,Cell type description,Processing logs,Title,...,age,treatment,genotype,lab,age.1,health state,cell_type,tissue_type,provider,sex
305,SRX356559,sa1,hg19,TFs and others,STAG1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"30569210,98.7,45.3,46902",GSM1233991: GM18486 SA1 1; Homo sapiens; ChIP-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,18486,NaN,NaN,NaN
366,SRX356783,sa1,hg19,TFs and others,STAG1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"39045039,98.3,44.9,50318",GSM1234215: GM2630 SA1 2; Homo sapiens; ChIP-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,2630,NaN,NaN,NaN
336,SRX356747,sa1,hg19,TFs and others,STAG1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"33660922,97.3,43.8,47950",GSM1234179: GM2588 SA1 2; Homo sapiens; ChIP-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,2588,NaN,NaN,NaN
321,SRX356729,sa1,hg19,TFs and others,STAG1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"32218948,97.4,46.1,42241",GSM1234161: GM2255 SA1 2; Homo sapiens; ChIP-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,2255,NaN,NaN,NaN
351,SRX356765,sa1,hg19,TFs and others,STAG1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"28293163,97.6,37.8,45107",GSM1234197: GM2610 SA1 2; Homo sapiens; ChIP-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,2610,NaN,NaN,NaN


#### what genomic bins are driving this component?

In [43]:
np.argsort(-v_dot_d_find_loci[:, component_idx])[:5]

array([298320, 298321,  54352, 114709,  46679])

In [44]:
v_dot_d_find_loci[np.argsort(-v_dot_d_find_loci[:, component_idx])[:5], component_idx]

array([4.89963650e-05, 4.77846741e-05, 4.77053255e-05, 4.73716488e-05,
       4.72138889e-05])

#### where is our loci of interest in this ranking?

In [45]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[genomic_bin_idx, component_idx])

41661

In [46]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[genomic_bin_idx, component_idx]) / v_dot_d_find_loci.shape[0]

0.1097667972630098

It's roughly on the top 10 percentile.


#### results of the enrichment analysis

In [47]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'HumanPhenotypeOntology').head(10)


,# ID,Desc,BPval,BFold
8,HP:0012103,Abnormality of the mitochondrion,1.981297e-08,2.430178
11,HP:0003287,Abnormality of mitochondrial metabolism,5.547277e-08,2.397021
16,HP:0010972,Anemia of inadequate production,1.467984e-07,2.084019
20,HP:0200042,Skin ulcer,2.149489e-07,2.021010
22,HP:0001581,Recurrent skin infections,3.267830e-07,2.644626
23,HP:0002665,Lymphoma,3.357733e-07,2.074043
27,HP:0005406,Recurrent bacterial skin infections,4.352149e-07,4.113836
37,HP:0006429,Broad femoral neck,2.049863e-06,7.213434
38,HP:0002722,Recurrent abscess formation,2.050315e-06,3.547911
43,HP:0001733,Pancreatitis,3.059544e-06,2.464570


In [48]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOBiologicalProcess').head(10)


,# ID,Desc,BPval,BFold
30,GO:0060333,interferon-gamma-mediated signaling pathway,9.010372e-11,2.559231
39,GO:0045047,protein targeting to ER,3.183959e-10,2.280882
44,GO:0071346,cellular response to interferon-gamma,5.468346e-10,2.159660
45,GO:0038096,Fc-gamma receptor signaling pathway involved i...,5.472179e-10,2.147846
47,GO:0002431,Fc receptor mediated stimulatory signaling pat...,5.672283e-10,2.146109
48,GO:0038094,Fc-gamma receptor signaling pathway,5.772810e-10,2.145260
57,GO:0072599,establishment of protein localization to endop...,2.336302e-09,2.173245
60,GO:0006614,SRP-dependent cotranslational protein targetin...,3.810641e-09,2.247171
65,GO:0015740,C4-dicarboxylate transport,7.328115e-09,4.436732
67,GO:0006613,cotranslational protein targeting to membrane,8.225698e-09,2.201283


In [49]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenotype').head(10)


,# ID,Desc,BPval,BFold
17,MP:0005153,abnormal B cell proliferation,3.181409e-20,2.120136
21,MP:0008217,abnormal B cell activation,3.436036e-18,2.012082
37,MP:0005093,decreased B cell proliferation,7.882232e-15,2.142833
41,MP:0008180,abnormal marginal zone B cell morphology,2.404651e-14,2.229240
45,MP:0008495,decreased IgG1 level,9.733961e-14,2.015448
46,MP:0001806,decreased IgM level,1.530640e-13,2.108597
50,MP:0002362,abnormal spleen marginal zone morphology,4.167194e-13,2.034597
56,MP:0008182,decreased marginal zone B cell number,1.844706e-12,2.315187
68,MP:0005154,increased B cell proliferation,1.483511e-10,2.424640
92,MP:0003303,peritoneal inflammation,1.156149e-08,3.763917


In [50]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenoSingleKO').head(10)


,# ID,Desc,BPval,BFold
29,MP:0005153,abnormal B cell proliferation,2.973987e-14,2.068460
40,MP:0005461,abnormal dendritic cell morphology,1.445606e-12,2.379902
42,MP:0001806,decreased IgM level,3.291375e-12,2.209544
53,MP:0005093,decreased B cell proliferation,1.718479e-10,2.145825
70,MP:0002418,increased susceptibility to viral infection,8.788483e-09,2.034574
72,MP:0000921,demyelination,1.427529e-08,2.140232
74,MP:0008125,abnormal dendritic cell number,1.474424e-08,2.382514
76,MP:0008088,abnormal T-helper 1 cell differentiation,2.115314e-08,3.146739
77,MP:0003303,peritoneal inflammation,2.189665e-08,4.346423
82,MP:0005154,increased B cell proliferation,4.557809e-08,2.279219


In [51]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOCellularComponent').head(10)


,# ID,Desc,BPval,BFold
9,GO:0005758,mitochondrial intermembrane space,4.658954e-08,2.924884
15,GO:0000307,cyclin-dependent protein kinase holoenzyme com...,1.661690e-06,2.979384
20,GO:0000788,nuclear nucleosome,3.649093e-06,6.005715
27,GO:0044798,nuclear transcription factor complex,1.326775e-05,2.009501
31,GO:0031095,platelet dense tubular network membrane,2.117509e-05,3.726373
32,GO:0000786,nucleosome,2.948377e-05,2.232604
34,GO:0031094,platelet dense tubular network,6.315071e-05,3.377614
35,GO:0005826,actomyosin contractile ring,6.359909e-05,4.808933
39,GO:0005761,mitochondrial ribosome,1.168505e-04,2.227518
40,GO:0090544,BAF-type complex,1.311371e-04,2.334366


In [52]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOMolecularFunction').head(10)


,# ID,Desc,BPval,BFold
4,GO:0008140,cAMP response element binding protein binding,2.734837e-09,6.534605
9,GO:0035673,oligopeptide transmembrane transporter activity,1.666693e-08,7.323736
11,GO:0015198,oligopeptide transporter activity,1.119388e-07,6.256376
13,GO:0004715,non-membrane spanning protein tyrosine kinase ...,4.323711e-07,2.190691
15,GO:0005313,L-glutamate transmembrane transporter activity,6.843716e-07,3.050932
16,GO:0015197,peptide transporter activity,7.138872e-07,4.969029
17,GO:0015556,C4-dicarboxylate transmembrane transporter act...,7.730462e-07,5.308462
19,GO:0005310,dicarboxylic acid transmembrane transporter ac...,1.089172e-06,2.631758
21,GO:0005154,epidermal growth factor receptor binding,1.510183e-06,2.286666
23,GO:0031996,thioesterase binding,1.584143e-06,3.249673


#### PC8 (the thrird component)


In [53]:
component_idx=8

#### what assays are driving this component?

In [54]:
np.argsort(-u_dot_d_find_assay[:, component_idx])[:5]

array([435, 434,  10,  76,  43])

In [55]:
u_dot_d_find_assay[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], component_idx]

array([0.1747739 , 0.13652367, 0.10681696, 0.04421963, 0.01690182])

In [56]:
metadata.iloc[np.argsort(-u_dot_d_find_assay[:, component_idx])[:5], :]

,sample_number,antibody,Genome assembly,Antigen class,Antigen,Cell type class,Cell type,Cell type description,Processing logs,Title,...,age,treatment,genotype,lab,age.1,health state,cell_type,tissue_type,provider,sex
435,SRX651493,NaN,hg19,Histone,H3K4me1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"141491413,92.6,2.4,104065",GSM1435518: LCL19240 H3K4me1; Homo sapiens; Ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,SRX651492,NaN,hg19,Histone,H3K4me1,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"112405175,94.2,1.6,121976",GSM1435516: LCL19239 H3K4me1; Homo sapiens; Ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,ERX329621,NaN,hg19,Unclassified,Unclassified,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"219047965,92.5,75.8,70997",Illumina HiSeq 2000 sequencing; Coordinated ef...,...,NaN,NaN,NaN,NaN,NaN,NaN,NA12891,NaN,NaN,male
76,ERX329687,NaN,hg19,Unclassified,Unclassified,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"238094924,94.2,33.9,144042",Illumina HiSeq 2000 sequencing; Coordinated ef...,...,NaN,NaN,NaN,NaN,NaN,NaN,NA12878,NaN,NaN,female
43,ERX329654,NaN,hg19,Unclassified,Unclassified,Blood,Lymphoblastoid cell line,Tissue=blood|Lineage=mesoderm|Description=pare...,"74449496,92.0,31.2,25320",Illumina HiSeq 2000 sequencing; Coordinated ef...,...,NaN,NaN,NaN,NaN,NaN,NaN,NA11831,NaN,NaN,male


#### what genomic bins are driving this component?

In [57]:
np.argsort(-v_dot_d_find_loci[:, component_idx])[:5]

array([308973,   4834, 347541, 186588,  52911])

In [58]:
v_dot_d_find_loci[np.argsort(-v_dot_d_find_loci[:, component_idx])[:5], component_idx]

array([6.19411917e-05, 5.91639390e-05, 5.76355494e-05, 5.75859867e-05,
       5.74937856e-05])

#### where is our loci of interest, chr17_38023 (index: 328517), in this ranking?

In [59]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[genomic_bin_idx, component_idx])

18886

In [60]:
np.sum(v_dot_d_find_loci[:, component_idx] >= v_dot_d_find_loci[genomic_bin_idx, component_idx]) / v_dot_d_find_loci.shape[0]

0.04976010496889664

It's roughly on the top 5 percentile.


#### results of the enrichment analysis

In [61]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'HumanPhenotypeOntology').head(10)


,# ID,Desc,BPval,BFold
1,HP:0006721,Acute lymphatic leukemia,0.000049,2.751787
4,HP:0006986,Upper limb spasticity,0.000110,8.260296
5,HP:0003584,Late onset,0.000120,3.540276
9,HP:0002047,Malignant hyperthermia,0.000234,2.987712
16,HP:0002365,Hypoplasia of the brainstem,0.000978,2.140287
17,HP:0003273,Hip contracture,0.001042,2.407612
19,HP:0001051,Seborrheic dermatitis,0.001150,2.652612
21,HP:0001650,Aortic valve stenosis,0.001179,2.379423
27,HP:0009141,Depletion of mitochondrial DNA in muscle tissue,0.001437,3.828807
31,HP:0001783,Broad metatarsal,0.001615,2.555863


In [62]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOBiologicalProcess').head(10)


,# ID,Desc,BPval,BFold
12,GO:0060334,regulation of interferon-gamma-mediated signal...,0.000004,2.610315
13,GO:0060330,regulation of response to interferon-gamma,0.000004,2.605978
17,GO:0006298,mismatch repair,0.000010,2.817783
20,GO:0034446,substrate adhesion-dependent cell spreading,0.000020,2.114061
23,GO:0010971,positive regulation of G2/M transition of mito...,0.000036,2.901271
29,GO:0046825,regulation of protein export from nucleus,0.000077,2.022195
32,GO:0010389,regulation of G2/M transition of mitotic cell ...,0.000107,2.032831
37,GO:0038030,non-canonical Wnt receptor signaling pathway v...,0.000162,2.849271
40,GO:2001300,lipoxin metabolic process,0.000180,3.613064
41,GO:0046031,ADP metabolic process,0.000181,4.668008


In [63]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenotype').head(10)


,# ID,Desc,BPval,BFold
0,MP:0002345,abnormal lymph node primary follicle morphology,9.052898e-07,3.167317
1,MP:0002252,abnormal oropharynx morphology,5.531696e-06,4.800082
2,MP:0010987,abnormal nephrogenic mesenchyme morphogenesis,1.072882e-05,2.618106
3,MP:0001705,abnormal proximal-distal axis patterning,1.763428e-05,2.080884
4,MP:0009323,abnormal spleen development,1.787223e-05,2.653792
6,MP:0009902,abnormal lateral nasal prominence morphology,6.414930e-05,2.968538
7,MP:0009904,tongue hypoplasia,7.173644e-05,2.336648
8,MP:0001704,abnormal dorsal-ventral axis patterning,8.530074e-05,2.011848
11,MP:0000481,abnormal enterocyte cell number,1.200177e-04,3.541474
16,MP:0011183,abnormal primitive endoderm morphology,2.305754e-04,2.291459


In [64]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'MGIPhenoSingleKO').head(10)


,# ID,Desc,BPval,BFold
0,MP:0002252,abnormal oropharynx morphology,0.000006,4.800082
2,MP:0002768,small adrenal glands,0.000034,2.283683
3,MP:0002345,abnormal lymph node primary follicle morphology,0.000035,3.245851
7,MP:0001704,abnormal dorsal-ventral axis patterning,0.000085,2.200372
8,MP:0000481,abnormal enterocyte cell number,0.000095,3.628263
9,MP:0009818,abnormal thromboxane level,0.000122,4.436419
10,MP:0004980,increased neuronal precursor cell number,0.000148,3.120505
11,MP:0008294,abnormal zona fasciculata morphology,0.000217,2.256848
12,MP:0002953,thick ventricular wall,0.000220,2.141185
13,MP:0011183,abnormal primitive endoderm morphology,0.000231,2.291459


In [65]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOCellularComponent').head(10)


,# ID,Desc,BPval,BFold
0,GO:0005771,multivesicular body,0.000006,2.350259
1,GO:0014731,spectrin-associated cytoskeleton,0.000035,3.243459
3,GO:0005605,basal lamina,0.000279,2.178926
8,GO:0001527,microfibril,0.000846,2.455440
10,GO:0019031,viral envelope,0.000912,4.117465
12,GO:0030992,intraflagellar transport particle B,0.001481,4.293770
13,GO:0033270,paranode region of axon,0.002383,2.286507
16,GO:0032300,mismatch repair complex,0.002978,2.594345
17,GO:0016235,aggresome,0.003720,2.026027
20,GO:0030990,intraflagellar transport particle,0.005258,2.526552


In [66]:
read_great_res_wrapper(enrichment_data_dir, component_idx, 'GOMolecularFunction').head(10)


,# ID,Desc,BPval,BFold
2,GO:0019206,nucleoside kinase activity,0.000005,3.109064
4,GO:0019789,SUMO ligase activity,0.000007,4.365886
5,GO:0042813,Wnt-activated receptor activity,0.000014,2.258498
10,GO:0004957,prostaglandin E receptor activity,0.000083,3.017609
15,GO:0004955,prostaglandin receptor activity,0.000476,2.373663
16,GO:0046625,sphingolipid binding,0.000506,2.495257
17,GO:0004954,prostanoid receptor activity,0.000557,2.343089
19,GO:0004017,adenylate kinase activity,0.000629,3.592177
20,GO:0004953,icosanoid receptor activity,0.000640,2.169161
23,GO:0008331,high voltage-gated calcium channel activity,0.000947,2.509615
